In [ ]:
import pygame, sys
from pygame.locals import *
import random, time

# Initialization
pygame.init()

# FPS settings
FPS = 60
FramePerSec = pygame.time.Clock()

# Colors
BLUE = (0, 0, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)

# Game parameters
SCREEN_WIDTH = 400
SCREEN_HEIGHT = 600
SPEED = 5  # Base speed
SCORE = 0
COINS = 0
COINS_FOR_SPEED_BOOST = 5  # Number of coins needed for speed boost

# Fonts
font = pygame.font.SysFont("Verdana", 60)
font_small = pygame.font.SysFont("Verdana", 20)
game_over = font.render("Game Over", True, BLACK)

# Load background
background = pygame.image.load("AnimatedStreet.png")

# Create screen
screen = pygame.display.set_mode((400, 600))
screen.fill(WHITE)
pygame.display.set_caption("Racer")

class Enemy(pygame.sprite.Sprite):
    #enemy moves
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("Enemy.png")
        self.rect = self.image.get_rect()
        self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), -100)
        
    def move(self):
        #enemy respawn
        global SCORE
        self.rect.move_ip(0, SPEED)
        if self.rect.top > SCREEN_HEIGHT:
            SCORE += 1
            self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), -100)

class Coin(pygame.sprite.Sprite):
    #coin class: values/weights
    def __init__(self, enemy_positions=[]):
        super().__init__()
        self.image = pygame.image.load("coin.png")
        self.image = pygame.transform.scale(self.image, (40, 40))
        self.rect = self.image.get_rect()
        
        # position so it's not close tro cars
        while True:
            pos = random.randint(40, SCREEN_WIDTH - 40), -100
            if not enemy_positions or not any(abs(pos[0] - e[0]) < 50 for e in enemy_positions):
                self.rect.center = pos
                break
        # random weight between 1-5 (heavier coins are rarer)
        self.weight = random.choices([1, 2, 3, 4, 5], weights=[40, 30, 20, 8, 2])[0]
        
    def move(self):
        #coin move and respawn
        self.rect.move_ip(0, SPEED)
        if self.rect.top > SCREEN_HEIGHT:
            self.kill()
            new_coin = Coin([e.rect.center for e in enemies])
            coins.add(new_coin)
            all_sprites.add(new_coin)

class Player(pygame.sprite.Sprite):
    #player care moves
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("Player.png")
        self.rect = self.image.get_rect()
        self.rect.center = (160, 520)

    def move(self):
        #movement key presses
        pressed_keys = pygame.key.get_pressed()
        if pressed_keys[K_LEFT] and self.rect.left > 0:
            self.rect.move_ip(-5, 0)
        if pressed_keys[K_RIGHT] and self.rect.right < SCREEN_WIDTH:
            self.rect.move_ip(5, 0)
        if pressed_keys[K_UP] and self.rect.top > 0:
            self.rect.move_ip(0, -5)
        if pressed_keys[K_DOWN] and self.rect.bottom < SCREEN_HEIGHT:
            self.rect.move_ip(0, 5)

# create sprites yahoo
P1 = Player()
E1 = Enemy()

# sprite groups
enemies = pygame.sprite.Group()
enemies.add(E1)
coins = pygame.sprite.Group()
all_sprites = pygame.sprite.Group()
all_sprites.add(P1)
all_sprites.add(E1)

# create first coin after enemies are initialized
C1 = Coin([E1.rect.center])
coins.add(C1)
all_sprites.add(C1)

# speed increase event
INC_SPEED = pygame.USEREVENT + 1
pygame.time.set_timer(INC_SPEED, 1000)

def game_over_screen():
   #display game over screen and handle restart/quit
    screen.fill(RED)
    screen.blit(game_over, (30, 250))
    final_score = font_small.render(f"Final Score: {SCORE}", True, WHITE)
    final_coins = font_small.render(f"Coins Collected: {COINS}", True, WHITE)
    instructions = font_small.render("Press SPACE to restart or ESC to quit", True, WHITE)
    screen.blit(final_score, (100, 350))
    screen.blit(final_coins, (100, 380))
    screen.blit(instructions, (30, 420))
    pygame.display.update()
    
    while True:
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == KEYDOWN:
                if event.key == K_SPACE:
                    return True
                elif event.key == K_ESCAPE:
                    return False

# main game loop
while True:
    for event in pygame.event.get():
        if event.type == INC_SPEED:
            SPEED += 0.2  # Gradual speed increase
        if event.type == QUIT:
            pygame.quit()
            sys.exit()

    # draw background
    screen.blit(background, (0, 0))
    
    # display scores
    scores = font_small.render(f"Score: {SCORE}", True, BLACK)
    screen.blit(scores, (10, 10))
    coin_text = font_small.render(f"Coins: {COINS}", True, BLACK)
    screen.blit(coin_text, (300, 10))
    speed_text = font_small.render(f"Speed: {SPEED:.1f}", True, BLACK)
    screen.blit(speed_text, (150, 10))

    # move and draw all objects
    for entity in all_sprites:
        screen.blit(entity.image, entity.rect)
        entity.move()

    # check collision with enemies
    if pygame.sprite.spritecollideany(P1, enemies):
        screen.fill(RED)
        screen.blit(game_over, (30, 250))
        pygame.display.update()
        time.sleep(2)
        if not game_over_screen():
            pygame.quit()
            sys.exit()
        else:
            # Reset game
            SCORE = 0
            COINS = 0
            SPEED = 5
            P1.rect.center = (160, 520)
            for enemy in enemies:
                enemy.rect.center = (random.randint(40, SCREEN_WIDTH - 40), -100)
            for coin in coins:
                coin.kill()
            new_coin = Coin([e.rect.center for e in enemies])
            coins.add(new_coin)
            all_sprites.add(new_coin)

    # collect coins
    collected_coins = pygame.sprite.spritecollide(P1, coins, True)
    for coin in collected_coins:
        COINS += coin.weight
        # Speed boost when collecting enough coins
        if COINS % COINS_FOR_SPEED_BOOST == 0:
            SPEED += 1.0  # Significant speed boost
        new_coin = Coin([e.rect.center for e in enemies])
        coins.add(new_coin)
        all_sprites.add(new_coin)

    pygame.display.update()
    FramePerSec.tick(FPS)